### Connect to a PostgreSQL Database from Python 
* Extracted CSV data into postgreSQL Database using SSIS
* To Transfer data from postgreSQL DataBase to python,connected postgreSQL DataBase to python using psycopg2 module



In [ ]:
!pip install psycopg2 #Install Psycopg2 using the pip command

In [6]:
import psycopg2 #import psycopg2 module to communicate with the PostgreSQL database.

#### To Connect to PostgreSQL in Python
 * Use the cursor() method
 * Create a cursor object using the connection object returned by the connect method to execute PostgreSQL queries from Python.

In [14]:
#connected postgreSQL DataBase to python using psycopg2 module
conn = psycopg2.connect(
   database="mydb", user='demo', password='Tata@123', host='127.0.0.1', port= '5432'
)
#Creating a cursor object using the cursor() method
cursor = conn.cursor()
print(conn)

<connection object at 0x000002A4DA46E040; dsn: 'user=demo password=xxx dbname=mydb host=127.0.0.1 port=5432', closed: 0>


In [17]:
import warnings 
warnings.filterwarnings("ignore")

In [23]:
import pandas as pd #importing pandas library

In [22]:
#Reading data using pandas from postgreSQL
data=pd.read_sql_query('''select * from data''',conn)
print(data)

      Country_Name Country_Code    Year        Value
0       Arab World          ARB  1960.0   92197753.0
1       Arab World          ARB  1961.0   94724510.0
2       Arab World          ARB  1962.0   97334442.0
3       Arab World          ARB  1963.0  100034179.0
4       Arab World          ARB  1964.0  102832760.0
...            ...          ...     ...          ...
15404     Zimbabwe          ZWE  2014.0   13586681.0
15405     Zimbabwe          ZWE  2015.0   13814629.0
15406     Zimbabwe          ZWE  2016.0   14030390.0
15407     Zimbabwe          ZWE  2017.0   14236745.0
15408     Zimbabwe          ZWE  2018.0   14439018.0

[15409 rows x 4 columns]


##### Create a  table in PostgreSQL Database using Python
* created table name “tbl_country”  in PostgreSQL.
* which contains column name countryname and countrycode
* committing our changes to the database using the commit() method.

### 2. Create table- tbl_Country (countryName, countryCode)
 

In [ ]:
# create table1-tbl_country 

cursor.execute("ROLLBACK") #undo the changes done in transactions
cursor.execute("create table tbl_country(countryname varchar(250),countrycode varchar(255))") #created table
conn.commit() #commiting changes in database

In [ ]:
#Inserted data in table name tbl_country from table name data

cursor.execute("ROLLBACK")
cursor.execute("""INSERT INTO tbl_country (countryname,countrycode)
                       SELECT "Country_Name","Country_Code"
                       FROM data""")

conn.commit()

In [20]:
#reading table name tbl_country 
table1=pd.read_sql_query('''
                       SELECT * FROM tbl_country 
                       ''',conn)
table1

,countryname,countrycode
0,Arab World,ARB
1,Arab World,ARB
2,Arab World,ARB
3,Arab World,ARB
4,Arab World,ARB
...,...,...
15404,Zimbabwe,ZWE
15405,Zimbabwe,ZWE
15406,Zimbabwe,ZWE
15407,Zimbabwe,ZWE


### 3. Calculate the percentage increase every year for each country and save into "tbl_population" with the fields (countryCode, year, population, percentIncrease)
    

In [12]:
# create table2 -tbl_population contain column names countrycode,year,population,percentIncrease
cursor.execute("ROLLBACK")
cursor.execute("create table tbl_population(countrycode varchar(255), year numeric(90),population numeric(90), percentIncrease float(50))")
conn.commit()

In [13]:
# Calculate the percentage increase every year for each country and save into "tbl_population" with the fields (countryCode, year, population, percentIncrease)

cursor.execute("ROLLBACK")
cursor.execute("""INSERT INTO tbl_population(countrycode,year,population,percentIncrease)
                  SELECT "Country_Code" as countrycode,"Year","Value" as population,
                  ("Value"-LAG("Value") OVER (PARTITION BY "Country_Code"
                   ORDER BY "Year" ASC))/LAG ("Value") OVER (PARTITION BY "Country_Code" 
                    ORDER BY "Year" ASC)*100 AS percentIncrease
                        FROM data""")



In [24]:
#Reading table name tbl_population 
#Here 4th column shows calculated percentage increase every year for each country and save into "tbl_population table
table2=pd.read_sql_query('''
                       SELECT * FROM tbl_population
                       ''',conn)
table2

,countrycode,year,population,percentincrease
0,ABW,1960.0,54211.0,NaN
1,ABW,1961.0,55438.0,2.263378
2,ABW,1962.0,56225.0,1.419604
3,ABW,1963.0,56695.0,0.835927
4,ABW,1964.0,57032.0,0.594409
...,...,...,...,...
15404,ZWE,2014.0,13586681.0,1.770178
15405,ZWE,2015.0,13814629.0,1.677731
15406,ZWE,2016.0,14030390.0,1.561830
15407,ZWE,2017.0,14236745.0,1.470772


### 1.The data transfer contain log file created with the number of records transferred to each table.

In [30]:
#while data transfering log file generated and that log file extracted into postgreSQL DataBase using SSIS and store that log file in table name tbl_logfile
log_file=pd.read_sql_query('''
                       select *
                              from tbl_logfile;
                       ''',conn)
print(log_file)

                           Event   computer                   operator  \
0                  OnPreValidate  NWLT22122  AzureAD\sankalpnaMahamuni   
1                  OnInformation  NWLT22122  AzureAD\sankalpnaMahamuni   
2                                                                        
3                      OnWarning  NWLT22122  AzureAD\sankalpnaMahamuni   
4                                                                        
..                           ...        ...                        ...   
476  OnPipelinePostComponentCall  NWLT22122  AzureAD\sankalpnaMahamuni   
477  OnPipelinePostComponentCall  NWLT22122  AzureAD\sankalpnaMahamuni   
478                OnPostExecute  NWLT22122  AzureAD\sankalpnaMahamuni   
479                   PackageEnd  NWLT22122  AzureAD\sankalpnaMahamuni   
480                                                                      

             source                                sourceid  \
0    Data Flow Task  {1473D0FB-8D46-4051-999E-54